In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras as K
print(tf.__version__)
print(tf.config.list_physical_devices())
!pwd

2.4.1
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
/content


In [ ]:
def recall_m(y_true, y_pred):
    K = tf.keras.backend
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    K = tf.keras.backend
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    K = tf.keras.backend
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [ ]:
%cd /
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

!ln -s /content/gdrive/My\ Drive/cv_final_project /gd
!ls /gd
!mkdir -p /w
%cd /w

/
Mounted at /content/gdrive
data_aug.7z		 v_3_xception_sgd.ipynb
Dataset.7z		 v_4_xception_adam.ipynb
v_1_resnet50_ft40.ipynb  v_5_xception_adam_lighting_aug.ipynb
v_2_resnet50.ipynb
/w


In [ ]:
!cp /gd/data_aug.7z /w
!7z x /w/data_aug.7z -o/w/Dataset


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /w/             1 file, 157905383 bytes (151 MiB)

Extracting archive: /w/data_aug.7z
--
Path = /w/data_aug.7z
Type = 7z
Physical Size = 157905383
Headers Size = 69525
Method = LZMA2:24
Solid = +
Blocks = 1

  0%      1% 4        3% 186 - data_aug/0/0183.jpg                                5% 375 - data_aug/0/0372.jpg                                7% 430 - data_aug/0/0427.jpg                                9% 557 - data_aug/0/0554.jpg                               11% 

In [ ]:
!ls Dataset/data_aug

0  1  2


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import preprocess_input

img_height, img_width = 224, 224
batch_size = 64
nb_epochs = 60
nb_classes = 3

train_data_dir = '/w/Dataset/data_aug'

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=0.25,
    width_shift_range=0.25,
    height_shift_range=0.25,
    zoom_range=0.2,
    rotation_range=40,
    validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

Found 5612 images belonging to 3 classes.
Found 1401 images belonging to 3 classes.


In [ ]:
from tensorflow.keras.applications import ResNet50, ResNet50V2, Xception
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, AveragePooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam, Adadelta, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

def build_model_fine_tune():
    baseModel = Xception(
        include_top=False,
        weights="imagenet",
        input_tensor=Input(shape=(img_height, img_width, 3)),
        pooling='avg',
    )
    
    # for layer in baseModel.layers:
    #     layer.trainable = False
    # for layer in baseModel.layers[-40:]:
    #     layer.trainable = True

    headModel = baseModel.output
    headModel = Dropout(0.2)(headModel)
    headModel = Dense(1024, activation="relu")(headModel)
    headModel = Dropout(0.2)(headModel)
    headModel = Dense(1024, activation="relu")(headModel)
    headModel = Dropout(0.2)(headModel)
    headModel = Dense(512, activation="relu")(headModel)
    headModel = Dropout(0.2)(headModel)
    headModel = Dense(512, activation="relu")(headModel)
    headModel = Dropout(0.2)(headModel)
    headModel = Dense(nb_classes, activation="softmax")(headModel)
    
    model = Model(inputs=baseModel.input, outputs=headModel)

    return model


learning_rate = 0.0001
weights = os.path.join('', 'weightsv5.h5')
callbacks = [ EarlyStopping(monitor='val_f1_m', patience=20, verbose=0, mode='max'), 
              ModelCheckpoint(weights, monitor='val_f1_m', save_best_only=True, verbose=1, mode='max'),
              ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=20, verbose=0, mode='min', min_delta=0.0001, cooldown=0, min_lr=0)]
loss = CategoricalCrossentropy()
# optimizer = SGD(lr=1e-2, momentum=0.9, decay=1e-2/40)
optimizer = Adam(lr=learning_rate)
model = build_model_fine_tune()
model.compile(
    loss=loss, 
    optimizer='adam', 
    metrics=['accuracy', f1_m, precision_m, recall_m],
)

history = model.fit(
    train_generator, 
    validation_data=validation_generator, 
    epochs=nb_epochs, 
    batch_size=batch_size, 
    callbacks=callbacks,
)

Epoch 1/60
88/88 [==============================] - 138s 2s/step - loss: 0.6045 - accuracy: 0.7412 - f1_m: 0.6972 - precision_m: 0.8286 - recall_m: 0.6174 - val_loss: 1.3199 - val_accuracy: 0.7966 - val_f1_m: 0.7967 - val_precision_m: 0.8052 - val_recall_m: 0.7885

Epoch 00001: val_f1_m improved from -inf to 0.79668, saving model to weightsv5.h5
Epoch 2/60
88/88 [==============================] - 128s 1s/step - loss: 0.2440 - accuracy: 0.8982 - f1_m: 0.8981 - precision_m: 0.9159 - recall_m: 0.8814 - val_loss: 0.7424 - val_accuracy: 0.8508 - val_f1_m: 0.8558 - val_precision_m: 0.8624 - val_recall_m: 0.8494

Epoch 00002: val_f1_m improved from 0.79668 to 0.85582, saving model to weightsv5.h5
Epoch 3/60
88/88 [==============================] - 128s 1s/step - loss: 0.1582 - accuracy: 0.9511 - f1_m: 0.9507 - precision_m: 0.9566 - recall_m: 0.9449 - val_loss: 0.5688 - val_accuracy: 0.8908 - val_f1_m: 0.8901 - val_precision_m: 0.8946 - val_recall_m: 0.8857

Epoch 00003: val_f1_m improved from

In [ ]:
best_model = tf.keras.models.load_model('/w/weightsv5.h5', custom_objects={'f1_m':f1_m, 'precision_m':precision_m, 'recall_m':recall_m})
best_model.compile(
    loss=loss, 
    optimizer='adam', 
    metrics=['accuracy', f1_m, precision_m, recall_m],
)

In [ ]:
test_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
  )
best_model.evaluate(test_generator, batch_size=batch_size)

Found 1401 images belonging to 3 classes.
22/22 [==============================] - 19s 834ms/step - loss: 0.0172 - accuracy: 0.9964 - f1_m: 0.9968 - precision_m: 0.9972 - recall_m: 0.9964


[0.01716098189353943,
 0.9964311122894287,
 0.9968011975288391,
 0.9971590638160706,
 0.9964488744735718]

In [ ]:
best_model.save("/w/modelv5.h5")

In [ ]:
download("/gd/modelv5.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

test_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
  )

Y_pred = best_model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = ['0', '1', '2']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

Found 1401 images belonging to 3 classes.
Confusion Matrix
[[951   6   1]
 [  2 147   1]
 [  0   0 293]]
Classification Report
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       958
           1       0.96      0.98      0.97       150
           2       0.99      1.00      1.00       293

    accuracy                           0.99      1401
   macro avg       0.98      0.99      0.99      1401
weighted avg       0.99      0.99      0.99      1401



In [ ]:
!cp /gd/Dataset.7z /w
!7z x /w/Dataset.7z -o/w/OrgDataset
!ls /w/OrgDataset


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /w/             1 file, 60678820 bytes (58 MiB)

Extracting archive: /w/Dataset.7z
--
Path = /w/Dataset.7z
Type = 7z
Physical Size = 60678820
Headers Size = 27545
Method = LZMA2:24
Solid = +
Blocks = 1

  0%      5% 3       10% 185 - 0/0183.jpg                      15% 374 - 0/0372.jpg                      18% 375 - 0/0373.jpg                      24% 556 - 0/0554.jpg                      27% 737 - 0/0735.jpg                      32% 737 - 0/0735.jpg                     

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import preprocess_input

img_height, img_width = 224, 224
batch_size = 64
nb_epochs = 60
nb_classes = 3

org_data_dir = '/w/OrgDataset'

validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    zoom_range=0.2,
    horizontal_flip=False,
    rotation_range=40,
    validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')